In [19]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import numpy as np
import joblib
import os
from xgboost import XGBRegressor

# Initialize the main window
root = tk.Tk()
root.title("Prediction of Load-Carrying Capacity")

# Create the main canvas
canvas = tk.Canvas(root, width=1000, height=2000, background='#FFFFFF')
canvas.pack()

# Function to create labels on the canvas
def create_label(text, font, fg, bg, x, y):
    label = tk.Label(root, text=text, font=font, fg=fg, bg=bg)
    canvas.create_window(x, y, anchor="w", window=label)
    return label

# Check if the XGBoost model file exists
model_path_xgb = os.path.expanduser("~/Desktop/XGB.joblib")
if not os.path.exists(model_path_xgb):
    messagebox.showerror("Error", "XGB model file not found.")
    raise SystemExit  # Exit if the model file is not found

# Load the XGBoost model
model_xgb = joblib.load(model_path_xgb)

# Define the GUI elements
label_inputdefinetitle2 = create_label('Prediction of Load-Carrying Capacity', ('Palatino Linotype', 18, 'bold', 'underline'), '#0000FF', '#FFFF00', 35,10)
label_inputdefinetitle = create_label('Definition of Parameters', ('Palatino Linotype', 16, 'bold', 'italic'), '#000000', '#FFFFFF', 10,45)
param_labels = [
    'X1: Concrete strength (fc’|MPa)',
    'X2: Height of column (H|mm)',
    'X3: Diameter of outer steel tube (from outer to outer) (2ao-ost|mm)',
    'X4: Diameter of outer steel tube (from inner to inner) (2ai-ost|mm)',
    'X5: Yield strength of outer steel tube (fy-ost|MPa)',
    'X6: Yield strength of longitudinal reinforcing rebars (fyv|MPa)',
    'X7: Ratio of longitudinal reinforcing rebars (ρv)',
    'X8: Volumetric ratio of circular/spiral reinforcing rebars (kes*ρh)',
    'X9: Yield strength of circular/spiral reinforcing rebars (fyh|MPa)',
    'X10: Total area of internal reinforcing steel section (Box,E,O,X,H) (As-iss|mm^2)',
    'X11: Yield strength of internal reinforcing steel section (fy-iss|MPa)',
    'X12: Eccentricity (e|mm)',
]
for i, text in enumerate(param_labels, start=1):
    create_label(text, ('Georgia', 13), '#006600', '#FFFFFF', 10, 50 + i*30)

label_inputs = create_label('Inputs', ('Palatino Linotype', 16, 'bold', 'italic'), '#000000', '#FFFFFF', 800, 45)

entry_fields = ['X1', 'X2', 'X3', 'X4','X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12']
entries = {}
for i, field in enumerate(entry_fields, start=1):
    label = create_label(f'{field}: ', ('Georgia', 14), '#FF0000', '#FFFFFF', 780, 45+ i*30)
    entry = tk.Entry(root)
    canvas.create_window(900, 45 + i*30, window=entry)
    entries[field] = entry

label_output = create_label('Output:', ('Palatino Linotype', 16, 'bold'), '#000000', '#FFFFFF', 20, 440)
label_result = create_label('Load carrying capacity', ('Georgia', 16), '#FF0000', '#FFFFFF', 100, 440)

# Predict button command function
def predict():
    input_values = []
    all_inputs_collected = True

    # Collect input data
    for field in entry_fields:
        value = entries[field].get()
        try:
            # Convert input to float and append if valid
            input_values.append(float(value))
        except ValueError:
            # Show error and mark input as incomplete
            messagebox.showerror("Error", f"Invalid input for {field}. Please enter a positive numeric value.")
            all_inputs_collected = False
            break

    # Proceed only if all inputs are valid
    if all_inputs_collected:
        if len(input_values) == 12:
            # Proceed with prediction using the collected input_values
            try:
                prediction_result = model_xgb.predict(np.array([input_values]))[0]
                label_result.config(text=       f'Load (kN)= {prediction_result:.2f}')
            except Exception as e:
                messagebox.showerror("Prediction Error", str(e))
        else:
            messagebox.showerror("Error", "All 12 inputs are required for the prediction.")

# Predict button
predict_button = tk.Button(root, text='Calculate', font=('Georgia', 12), command=predict, fg='#FFFFFF', bg='green')
canvas.create_window(450, 440, window=predict_button)

# Specify the image path
image_path = r"C:\Users\asus1\Desktop\image.png"  # Use a raw string for the file path

# Load the image from the specified path and create an image on the canvas using PIL
try:
    image = Image.open(image_path)
    root.image_image = ImageTk.PhotoImage(image)  # Store the image
    canvas.create_image(500, 410, image=root.image_image, anchor="n")
except Exception as e:
    messagebox.showerror("Error", f"Failed to load image: {e}")
    
# Run the main loop
root.mainloop()